<a href="https://colab.research.google.com/github/dTony33/Pdb/blob/master/pdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Levenshtein

In [54]:
import re
import string
import sys
import heapq
import logging
import pandas as pd

from functools import partial
from Levenshtein import *

class StringMatcher:

    def _reset_cache(self):
        self._ratio = self._distance = None
        self._opcodes = self._editops = self._matching_blocks = None

    def __init__(self, isjunk=None, seq1='', seq2=''):
        if isjunk:
            warn("isjunk not NOT implemented, it will be ignored")
        self._str1, self._str2 = seq1, seq2
        self._reset_cache()

    def set_seqs(self, seq1, seq2):
        self._str1, self._str2 = seq1, seq2
        self._reset_cache()

    def set_seq1(self, seq1):
        self._str1 = seq1
        self._reset_cache()

    def set_seq2(self, seq2):
        self._str2 = seq2
        self._reset_cache()

    def ratio(self):
      if not self._ratio:
        self._ratio = ratio(self._str1, self._str2)
      return self._ratio    

    def get_matching_blocks(self):
      if not self._matching_blocks:
        self._matching_blocks = matching_blocks(self.get_opcodes(),self._str1, self._str2)  

In [55]:

df = pd.read_csv('/content/bookings_data.csv')

In [82]:
def partial_ratio(s1, s2):

    s1, s2 = utils.make_type_consistent(s1, s2)

    if len(s1) <= len(s2):
        shorter = s1
        longer = s2
    else:
        shorter = s2
        longer = s1

    blocks = get_matching_blocks(None, shorter, longer)
    # blocks = m.get_matching_blocks()

    scores = []
    for block in blocks:
        long_start = block[1] - block[0] if (block[1] - block[0]) > 0 else 0
        long_end = long_start + len(shorter)
        long_substr = longer[long_start:long_end]

        m2 = SequenceMatcher(None, shorter, long_substr)
        r = m2.ratio()
        if r > .995:
            return 100
        else:
            scores.append(r)

    return utils.intr(100 * max(scores))

In [83]:
confidenceLevels = pd.DataFrame({
    'wordLength' : [1,2,3,4,5,6,7,8,9,10], 
    'minRatio' : [100,100,100,100,80,80,80,80,75,75]
})

In [84]:
class StringProcessor(object):
  regex = re.compile(r"(?ui)\W")
  @classmethod
  def replace_non_letters_non_numbers_with_whitespace(cls, a_string):
    
    return cls.regex.sub(" ", a_string)

    strip = staticmethod(string.strip)
    to_lower_case = staticmethod(string.lower)
    to_upper_case = staticmethod(string.upper)

In [85]:
def asciidammit(s):
    if type(s) is str:
        return asciionly(s)
    elif type(s) is unicode:
        return asciionly(s.encode('ascii', 'ignore'))
    else:
        return asciidammit(unicode(s))

In [86]:
def full_process(s, force_ascii=False):
  if force_ascii:
    s = asciidammit(s)
    # Keep only Letters and Numbers (see Unicode docs).
  string_out = StringProcessor.replace_non_letters_non_numbers_with_whitespace(s)
    # Force into lowercase.
  string_out = StringProcessor.to_lower_case(string_out)
    # Remove leading and trailing whitespaces.
  string_out = StringProcessor.strip(string_out)
  return string_out

In [87]:
def validate_string(s):
    try:
        return len(s) > 0
    except TypeError:
        return False

In [88]:
def _token_set(s1, s2, partial=True, force_ascii=True, full_process=True):

    if not full_process and s1 == s2:
        return 100

    p1 = full_process(s1, force_ascii=force_ascii) if full_process else s1
    p2 = full_process(s2, force_ascii=force_ascii) if full_process else s2

    if not validate_string(p1):
        return 0
    if not validate_string(p2):
        return 0

    # pull tokens
    tokens1 = set(p1.split())
    tokens2 = set(p2.split())

    intersection = tokens1.intersection(tokens2)
    diff1to2 = tokens1.difference(tokens2)
    diff2to1 = tokens2.difference(tokens1)

    sorted_sect = " ".join(sorted(intersection))
    sorted_1to2 = " ".join(sorted(diff1to2))
    sorted_2to1 = " ".join(sorted(diff2to1))

    combined_1to2 = sorted_sect + " " + sorted_1to2
    combined_2to1 = sorted_sect + " " + sorted_2to1

    # strip
    sorted_sect = sorted_sect.strip()
    combined_1to2 = combined_1to2.strip()
    combined_2to1 = combined_2to1.strip()

    if partial:
        ratio_func = partial_ratio
    else:
        ratio_func = ratio

    pairwise = [
        ratio_func(sorted_sect, combined_1to2),
        ratio_func(sorted_sect, combined_2to1),
        ratio_func(combined_1to2, combined_2to1)
    ]
    return max(pairwise)


def token_set_ratio(s1, s2, force_ascii=True, full_process=True):
    return _token_set(s1, s2, partial=False, force_ascii=force_ascii, full_process=full_process)

In [89]:
def cal_ratio(s1, s2):
    # s1, s2 = utils.make_type_consistent(s1, s2)

    m = ratio(s1, s2)
    return round(100 * m)

In [90]:
cal_ratio('fuzzy wuzzy was a bear','wuzzy fuzzy was a bear')

91

In [91]:
def extractWithoutOrder(query, choices, scorer, score_cutoff=0):
    def no_process(x):
        return x
    processed_query = query
    pre_processor = no_process
    scorer = partial(scorer)
    try:
        # See if choices is a dictionary-like object.
        for key, choice in choices.items():
            processed = pre_processor(choice)
            score = scorer(processed_query, processed)
            if score >= score_cutoff:
                yield (choice, score, key)
    except AttributeError:
        # It's a list; just iterate over it.
        for choice in choices:
            processed = pre_processor(choice)
            score = scorer(processed_query, processed)
            if score >= score_cutoff:
                yield (choice, score)
def extractOne(query, choices,  scorer, score_cutoff=0):

    best_list = extractWithoutOrder(query, choices, scorer, score_cutoff)
    try:
        return max(best_list, key=lambda i: i[1])
    except ValueError:
        return None                

In [92]:
def FuzzyED(token = None, confidence_levels = True, default_level = 85):
    assert isinstance(token, str), "Tokens can be str() type only"
    if confidence_levels:
        for i, row in confidenceLevels.iterrows():
            if len(token) > confidenceLevels['wordLength'].max():
                min_ratio = confidenceLevels['minRatio'].min()
            else:
                min_ratio = confidenceLevels.loc[confidenceLevels['wordLength'] == len(token)]['minRatio'].values[0]
    else:
        min_ratio = default_level
    # print(min_ratio)    
    return int(min_ratio)

In [93]:
def Similarity(token = None, dictionary = None, min_ratio = None):
    assert isinstance(token, str)
    assert isinstance(dictionary, dict) 
    assert isinstance(min_ratio, int)
    for key, values in dictionary.items():
        match = extractOne(token, values, scorer = cal_ratio)
        if min_ratio <= match[1]:
            return (match + (key, ))

In [94]:
tokens = ["insurence", "party", "libertey"]
dictionary = {
    'category1' : ['insurance', 'policies', 'worldbank'],
    'category2' : ['liberty', 'freedom', 'helloworld', 'house']
}

for token in tokens:
    fuzzy_ratio = FuzzyED(token = token, confidence_levels = True)
    similarity_score = Similarity(token = token, dictionary = dictionary, min_ratio = fuzzy_ratio)
    if not similarity_score == None:
        print("'" + token + "'" + " matches with " + str(similarity_score[0]) + "[" + similarity_score[2] + "]" + " with a score of: " + str(similarity_score[1]))

'insurence' matches with insurance[category1] with a score of: 89
'libertey' matches with liberty[category2] with a score of: 93


In [95]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]
extractOne('cowboys',choices,cal_ratio,2)

('Dallas Cowboys', 57)

In [98]:
def get_ratio(row):
    name = row['Expedia']
    name1 = row['Booking.com']
    return cal_ratio(name, name1)

df[df.apply(get_ratio, axis=1) > 70].head(10)

,Expedia,Booking.com
2,"Grand Corner King Room, 1 King Bed",Grand Corner King Room
4,"High-Floor Premium Room, 1 King Bed",High-Floor Premium King Room
9,"Room, 2 Double Beds (19th to 25th Floors)",Two Double Beds - Location Room (19th to 25th ...
11,Deluxe Room,Deluxe Double Room
13,"Signature Room, 2 Queen Beds",Signature Two Queen
19,"Club Room, Premium 2 Queen Beds",Club Premium Two Queen
25,"Deluxe Room, 1 King Bed",Deluxe Room - One King Bed
26,"Deluxe Room, 2 Queen Beds",Deluxe Room - Two Queen Beds
45,"Business Double Room, 2 Double Beds",Business Double Room With Two Double Beds
46,"Room, 2 Queen Beds, City View",Queen Room With Two Queen Beds and City View


In [80]:
score = len(df[df.apply(get_ratio, axis=1) > 70]) / len(df)

In [77]:
score

0.22330097087378642